In [2]:
import pandas as pd
import time
import joblib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
df = pd.read_pickle(r'D:\JupyterNotebook\DataProcess\preprocess_SWsastrawi_stemming.pkl')
dfNo = pd.read_pickle(r'D:\JupyterNotebook\DataProcess\preprocess_SWsastrawi_nostemming.pkl')

In [7]:
kf = KFold(n_splits=5)
i = 1
for train_index, test_index in kf.split(df):
#     print(test_index)
#     print(train_index[0],"-",train_index[len(train_index)-1])
    print(test_index[0],"+",test_index[len(test_index)-1])
    X_train = df['text'].iloc[train_index[0]:train_index[len(train_index)-1]]
    X_test = df['text'].iloc[test_index[0]:test_index[len(test_index)-1]]
    y_train = df['label'].iloc[train_index[0]:train_index[len(train_index)-1]].astype('int')
    y_test = df['label'].iloc[test_index[0]:test_index[len(test_index)-1]].astype('int')
    
#     vectorizer = TfidfVectorizer()
#     X_train = vectorizer.fit_transform(X_train)
#     X_test = vectorizer.transform(X_test)
    counter = CountVectorizer(min_df=3,
                              dtype=np.double)
    counter.fit(X_train)
    X_train = counter.transform(X_train)
    X_test = counter.transform(X_test)
    
    transformer = TfidfTransformer(norm='l2', sublinear_tf=True)
    X_train = transformer.fit_transform(X_train)
    X_test = transformer.transform(X_test)
    
    X_train,X_test = select_features(X_train, y_train, X_test)
    
    modelknn2 = KNeighborsClassifier(n_neighbors=13, weights='uniform', metric='euclidean')
    modelknn2.fit(X_train,y_train)
    y_pred = modelknn2.predict(X_test)
#     print(classification_report(y_test,y_pred))
    result = modelknn2.score(X_test, y_test)
    print("Fold ke-",i,"=",result)
    print("----------------")
    i+=1

0 + 191
Fold ke- 1 = 0.3717277486910995
----------------
192 + 383
Fold ke- 2 = 0.6073298429319371
----------------
384 + 575
Fold ke- 3 = 0.7329842931937173
----------------
576 + 767
Fold ke- 4 = 0.7958115183246073
----------------
768 + 959
Fold ke- 5 = 0.612565445026178
----------------


In [4]:
def split(strt,fnsh,df):
    X_test = df['text'].iloc[strt:fnsh]
    y_test = df['label'].iloc[strt:fnsh].astype('int')
    df.drop(df.index[strt:fnsh])
    X_train = df['text']
    y_train = df['label'].astype('int')    
    return X_train, X_test, y_train, y_test

In [5]:
def select_features(train_X, train_y, test_X):
#     total fitur 7343
# 2004
# 1802 nltk
# 2599 sastrawi 1967
    selector = SelectKBest(chi2, k=1967)
    selector.fit(train_X, train_y)
#     new = selector.fit_transform(train_X, train_y)
#     print(new.shape)
    train_X = selector.transform(train_X)
    test_X = selector.transform(test_X)
    return train_X, test_X

In [6]:
def classifier(df,distance_name,k_neighbor):
    print("Nama distance=",distance_name)
    print("Jumlah K tetangga=",k_neighbor)
    spl = [0,36,72,108,144,180,216,252,288,324,360]
    tempspl = []
    i = 0
    akurasimax=0
    while (i<len(spl)-1):
        if len(spl)-2 != len(tempspl):
            X_train, X_test, y_train, y_test = split(spl[i],spl[i+2],df)
            tempspl.append(spl[i])
    #   ketika array[i+2] == null (looping terakhir)
        elif (len(spl)-len(tempspl)) == 2:
    #       split last and first
            X_test1 = df['text'].iloc[spl[i]:spl[i+1]]
            y_test1 = df['label'].iloc[spl[i]:spl[i+1]].astype('int')
            X_test2 = df['text'].iloc[tempspl[0]:tempspl[1]]
            y_test2 = df['label'].iloc[tempspl[0]:tempspl[1]].astype('int')
    #       concat last and first
            X_test = pd.concat([X_test1, X_test2], axis=0)
            y_test = pd.concat([y_test1, y_test2], axis=0)
    #       split except last and first
            X_train = df['text'].iloc[tempspl[1]:tempspl[len(tempspl)-1]]
            y_train = df['label'].iloc[tempspl[1]:tempspl[len(tempspl)-1]].astype('int')
        
        vectorizer = TfidfVectorizer(min_df=3, dtype=np.double, norm='l2', sublinear_tf=True)
        X_train = vectorizer.fit_transform(X_train)
        X_test = vectorizer.transform(X_test)

#         counter = CountVectorizer(min_df=3,
#                               dtype=np.double)
#         counter.fit(X_train)
#         X_train = counter.transform(X_train)
#         X_test = counter.transform(X_test)

#         transformer = TfidfTransformer(norm='l2', sublinear_tf=True)
#         X_train = transformer.fit_transform(X_train)
#         X_test = transformer.transform(X_test)
        
#         X_train,X_test = select_features(X_train, y_train, X_test)

        modelknn = KNeighborsClassifier(n_neighbors=k_neighbor, weights='uniform', metric=distance_name)
        modelknn.fit(X_train,y_train)
        y_pred = modelknn.predict(X_test)
#         print(classification_report(y_test,y_pred))
        result = modelknn.score(X_test, y_test)
        print("Fold ke-",i+1,"=",result)
        if (result >= akurasimax):
            akurasimax = result
            foldke = i+1
        i+=1
    print("Akurasi terbaik adalah",akurasimax,"pada fold ke-",foldke)
    print("------------------------------------")
    return akurasimax

In [22]:
%%time
# %%capture cap --no-stderr
start_time = time.time()
dfcount = [df,dfNo]
distancecount = ['euclidean','manhattan','minkowski']
kcount = [1,3,5,7,9,11,13,15]
for x in range(2):
    arrd1 = []
    arrd2 = []
    arrd3 = []
    for y in range(len(distancecount)):
        maxresult = []
        for z in range(len(kcount)):
            if x==0:
                print('Dengan Stemming')
            else:
                print('Tanpa Stemming')
            tempmax = classifier(dfcount[x],distancecount[y],kcount[z])
            maxresult.append(tempmax)
            if y==0:
                arrd1.append(tempmax)
            elif y==1:
                arrd2.append(tempmax)
            elif y==2:
                arrd3.append(tempmax)
#         tapi fold yang dipakai beda beda
    plt.plot(kcount, arrd1, label = "euclidean")
    plt.plot(kcount, arrd2, label = "manhattan")
    plt.plot(kcount, arrd3, label = "minkowski")
    plt.xticks(kcount)
    if x==0:
        plt.title("Stemming")
    else:
        plt.title("Tidak Stemming")
    plt.xlabel('Fold')
    plt.ylabel('Akurasi')
    plt.legend()
    plt.pause(0.05)
plt.show
print("Lama Proses %s seconds" % (time.time() - start_time))
# with open('5000_fitur_SWnltkv2.txt', 'w') as f:
#     f.write(cap.stdout)

Dengan Stemming
Nama distance= euclidean
Jumlah K tetangga= 1
Fold ke- 1 = 1.0
Fold ke- 2 = 1.0
Fold ke- 3 = 1.0
Fold ke- 4 = 1.0
Fold ke- 5 = 1.0
Fold ke- 6 = 1.0
Fold ke- 7 = 1.0
Fold ke- 8 = 1.0
Fold ke- 9 = 1.0
Fold ke- 10 = 0.125
Akurasi terbaik adalah 1.0 pada fold ke- 9
------------------------------------
Dengan Stemming
Nama distance= euclidean
Jumlah K tetangga= 3
Fold ke- 1 = 0.8333333333333334
Fold ke- 2 = 0.8611111111111112
Fold ke- 3 = 0.7916666666666666
Fold ke- 4 = 0.8194444444444444
Fold ke- 5 = 0.8194444444444444
Fold ke- 6 = 0.7916666666666666
Fold ke- 7 = 0.875
Fold ke- 8 = 0.9166666666666666
Fold ke- 9 = 0.9027777777777778
Fold ke- 10 = 0.6111111111111112
Akurasi terbaik adalah 0.9166666666666666 pada fold ke- 8
------------------------------------
Dengan Stemming
Nama distance= euclidean
Jumlah K tetangga= 5
Fold ke- 1 = 0.8333333333333334
Fold ke- 2 = 0.8194444444444444
Fold ke- 3 = 0.8194444444444444
Fold ke- 4 = 0.8472222222222222
Fold ke- 5 = 0.819444444444444

KeyboardInterrupt: 

# try--

In [ ]:
X_train = df['text'].iloc[72:]
X_test = df['text'].iloc[:72]
y_train = df['label'].iloc[72:].astype('int')
y_test = df['label'].iloc[:72].astype('int')

In [14]:
X = df['text']
y = df['label'].astype('int')
# X = dfNo['text']
# y = dfNo['label'].astype('int')

In [15]:
# Pembentukan tf-idf untuk pembobotan
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

# X_train = vectorizer.fit_transform(X_train)
# X_test = vectorizer.transform(X_test)

In [5]:
print(X_train.shape)

NameError: name 'X_train' is not defined

In [6]:
print(vectorizer.vocabulary_)
# print(len(vectorizer.get_feature_names()))

{'menpora': 8674, 'dukung': 3896, 'pianis': 10856, 'cilik': 2523, 'asal': 1011, 'kendal': 6320, 'yang': 14656, 'ingin': 5255, 'pecahkan': 9965, 'rekor': 11493, 'dunia': 3905, 'di': 2955, 'inggris': 5252, 'imam': 5141, 'nahrawi': 9275, 'hari': 4801, 'kamis': 5837, '16': 90, 'siang': 12556, 'kemarin': 6259, 'menerima': 8267, 'jefri': 5563, 'setiawan': 12508, 'akan': 612, 'melakukan': 7721, 'pemecahan': 10127, 'bermain': 1822, 'piano': 10857, 'dengan': 2874, 'mata': 7639, 'tertutup': 13755, 'bangga': 1290, 'dan': 2747, 'sepenuhnya': 12405, 'keinginan': 6134, 'untuk': 14231, 'bisa': 2105, 'mengharumkan': 8463, 'nama': 9288, 'indonesia': 5204, 'pentas': 10450, 'hal': 4748, 'itu': 5423, 'disampaikan': 3585, 'saat': 11805, 'jumpa': 5712, 'pers': 10720, 'loby': 7306, 'kantor': 5874, 'kemenpora': 6282, 'jakarta': 5474, 'dalam': 2734, 'audiensi': 1106, 'sebelumnya': 12100, 'menyampaikan': 8740, 'harapannya': 4791, 'agar': 547, 'menteri': 8688, 'pemuda': 10167, 'olahraga': 9597, 'dapat': 2773, 'm

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [17]:
%%time
modelknn2 = KNeighborsClassifier(n_neighbors=11, weights='uniform', metric='euclidean')
modelknn2.fit(X_train,y_train)

Wall time: 1.21 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='uniform')

In [ ]:
filename = 'D:\JupyterNotebook\Model\model_nostemming_173_splitmanual.sav'
joblib.dump(modelknn2, filename)

In [ ]:
filename = 'D:\JupyterNotebook\Model\model_stemming.sav'
loaded_model = joblib.load(filename)
# result = loaded_model.score(X_test, y_test)
# print(result)

In [18]:
y_pred = modelknn2.predict(X_test)
y_pred

array([ 1,  7,  8,  9,  0,  4,  6,  7,  6,  0,  1,  1,  2,  4,  7,  4,  4,
        2,  7, 11,  3,  2,  2,  5,  7,  3,  0,  0,  5,  8,  3,  4,  2,  5,
        1,  1,  4,  8,  3,  8,  8, 11,  7,  8, 10,  6,  7, 11,  2,  2, 10,
        3,  0,  3,  1,  6,  7,  4, 11,  1,  9,  0,  7,  8,  6,  9,  0,  4,
        8,  3, 11,  1,  1,  4])

In [ ]:
y_pred = loaded_model.predict(X_test)
y_pred

In [19]:
modelknn2.predict_proba(X_test)

array([[0.09090909, 0.27272727, 0.        , 0.        , 0.        ,
        0.        , 0.09090909, 0.27272727, 0.        , 0.        ,
        0.27272727, 0.        ],
       [0.09090909, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.90909091, 0.        , 0.        ,
        0.        , 0.        ],
       [0.18181818, 0.18181818, 0.09090909, 0.09090909, 0.09090909,
        0.        , 0.09090909, 0.        , 0.27272727, 0.        ,
        0.        , 0.        ],
       [0.09090909, 0.        , 0.        , 0.        , 0.18181818,
        0.        , 0.09090909, 0.18181818, 0.        , 0.45454545,
        0.        , 0.        ],
       [0.36363636, 0.09090909, 0.        , 0.09090909, 0.        ,
        0.        , 0.        , 0.09090909, 0.        , 0.18181818,
        0.        , 0.18181818],
       [0.        , 0.09090909, 0.        , 0.        , 0.45454545,
        0.27272727, 0.        , 0.        , 0.        , 0.        ,
        0.        ,

In [15]:
f1score_, recall_, precision_, cm_ = evaluation(d_test.label, y_pred_sw, label)
print(" f1-score: {}\n precision: {}\n recall: {}".format(f1score, precision, recall))

NameError: name 'evaluation' is not defined

In [20]:
print(confusion_matrix(y_test, y_pred))

[[3 0 1 0 0 0 0 1 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0 0 0]
 [0 0 0 6 0 0 1 0 0 0 0 0]
 [0 0 0 0 6 1 0 0 0 1 0 1]
 [1 1 1 0 3 2 0 1 0 0 0 0]
 [1 2 0 0 0 0 3 0 0 0 0 0]
 [1 0 0 0 0 0 0 6 1 0 0 0]
 [1 1 0 0 0 0 0 0 7 0 0 0]
 [0 1 0 1 0 0 1 0 0 2 0 0]
 [0 2 0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 1 0 0 0 4]]


In [21]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.43      0.60      0.50         5
           1       0.22      1.00      0.36         2
           2       0.71      1.00      0.83         5
           3       0.86      0.86      0.86         7
           4       0.67      0.67      0.67         9
           5       0.67      0.22      0.33         9
           6       0.60      0.50      0.55         6
           7       0.67      0.75      0.71         8
           8       0.88      0.78      0.82         9
           9       0.67      0.40      0.50         5
          10       1.00      0.50      0.67         4
          11       0.80      0.80      0.80         5

    accuracy                           0.65        74
   macro avg       0.68      0.67      0.63        74
weighted avg       0.71      0.65      0.65        74



In [22]:
result = modelknn2.score(X_test, y_test)
print(result)

0.6486486486486487


In [24]:
result_class = pd.DataFrame({'Actual Result':y_test, 'Predicted Result':y_pred})
result_class

     Actual Result  Predicted Result
309             10                 1
218              7                 7
260              8                 8
288              9                 9
5                0                 0
..             ...               ...
119              3                 3
356             11                11
300              9                 1
255              8                 1
179              5                 4

[74 rows x 2 columns]


In [25]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(result_class)

     Actual Result  Predicted Result
309             10                 1
218              7                 7
260              8                 8
288              9                 9
5                0                 0
166              5                 4
184              6                 6
181              5                 7
195              6                 6
153              5                 0
48               1                 1
47               1                 1
79               2                 2
150              4                 4
358             11                 7
124              4                 4
145              4                 4
82               2                 2
215              7                 7
146              4                11
104              3                 3
72               2                 2
84               2                 2
125              4                 5
231              7                 7
93               3                 3
1